In [58]:
import pandas as pd
import numpy as np

In [59]:
# 预处理，补充缺失的时间点，然后再对缺失的时间点用前面或者后面的值填充
def get_timeseries(df, time_col):
    df[time_col] = pd.to_datetime(df[time_col])
    ts_series = df[time_col].drop_duplicates()
    ts_series.sort_values(inplace=True)
    return ts_series


def get_freq(ts_series):
    """
    :param ts_series(series):
    :return:
    """
    ts_diff = ts_series.diff().dropna()
    ts_most_appear = int(ts_diff.value_counts().index[0].total_seconds())  # in seconds
    return ts_most_appear
    # return "{}s".format(ts_most_appear)


def get_real_index(df, time_col):
    """
    :param df:
    :param time_col:
    :return:
    """
    ts_series = get_timeseries(df, time_col)
    f_int = get_freq(ts_series)
    f = "{}s".format(f_int)
    st, ed = ts_series.min(), ts_series.max()
    real_index = pd.date_range(st, ed, freq=f)
    return real_index


def add_time_idx(df, time_col):
    assert len(df) != 0, "pandas data length is zero"
    assert time_col is not None, "time col should be given"
    real_time = get_real_index(df, time_col)
    df_real_time = pd.DataFrame(real_time, columns=[time_col])
    df_real_time.reset_index(inplace=True)
    df_real_time.rename(columns={"index": "time_idx"}, inplace=True)
    df_real_time["time_idx"] = df_real_time.time_idx.astype("int")
    df_format = pd.merge(df, df_real_time, on='collect_time', how="right")
    return df_format

def random_dates(start, end, n=1):
    start_u = start.value//10**9
    print(start_u)
    end_u = end.value//10**9
    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='s')

# #调用add_time_idx，data是输入的dataframe
# data_add_time = add_time_idx(data, "collect_time")
# print('nan before fill:\n',data_all.isna().sum())

# #forward and backward fill 前向后向填充
# data_fill_f = data_add_time.fillna(method='ffill')
# data_fill = data_fill_f.fillna(method='bfill')
# print('nan after fill:\n',data_fill.isna().sum(),data_fill.shape)

In [60]:
# 删除异常数据点
def delete_outliers(data, point_name):
    # print(data.isna().sum())
    if point_name in ['回风温度', '送风温度', '水路进水温度']:
        data[(data<0)|(data>60)] = np.nan
    if point_name == '风机转速':
        data[(data<0)|(data>2000)] = np.nan
    if point_name == '水阀开度':
        data[(data<0)|(data>100)] = np.nan   
    if point_name == '冷通道温度':
        data[(data<0.5)|(data>80)] = np.nan
    # if point_name in ['系统故障状态', '风机运行状态', '制冷状态', '远程关机', '系统运行状态']:
    #     data[(data!=0)|(data!=1)] = np.nan
    # print(data.isna().sum())
    return data

In [61]:
# point_name = '冷通道温度'
# df = pd.read_csv("./data_folder/data_{}.csv".format(0))
# data_i = df[df['device_cmdb_id']=='sensor_{}'.format(18)][['collect_time','point_value']]
# new_name = 'sensor_{}_'.format(18) + point_name
# data_i.rename(columns = {'point_value':new_name}, inplace = True)
# data_add_time = add_time_idx(data_i, "collect_time")  # add time label
# print('data_add_time:\n',data_add_time.isna().sum())
# data_add_time[new_name] = delete_outliers(data_add_time[new_name], point_name=point_name) # delete outliers
# print('after delete_outliers:\n',data_add_time.isna().sum())

In [62]:
all_point_name = ['回风温度', '送风温度', '水路进水温度', '系统运行状态', '系统故障状态', '风机运行状态', '风机转速', '制冷状态', '水阀开度', '远程关机']
# all_point_name = ['回风温度', '送风温度', '水路进水温度', '风机转速', '水阀开度'] # continious

def collect_point(k, s_init=18, s_end=35, c_init=20, c_end=39):
    df = pd.read_csv("./data_folder/data_{}.csv".format(k))

    # sensor part
    for i in range(s_init,s_end+1):
        point_name = '冷通道温度'
        data_i = df[df['device_cmdb_id']=='sensor_{}'.format(i)][['collect_time','point_value']]
        new_name = 'sensor_{}_'.format(i) + point_name
        data_i.rename(columns = {'point_value':new_name}, inplace = True)
        data_add_time = add_time_idx(data_i, "collect_time")                           # add time label
        data_add_time[new_name] = delete_outliers(data_add_time[new_name], point_name) # delete outliers
        if i==s_init:
            data_all = data_add_time[['collect_time',new_name.format(i)]]
        else:
            data_all = pd.concat([data_all, data_add_time[new_name]], axis=1)

    # crah part
    point_num = len(all_point_name)
    crah_num = c_end+1-c_init
    for j in range(point_num):
        point_name = all_point_name[j]
        for i in range(crah_num):
            data_ij = df[(df['device_cmdb_id']=='crah_{}'.format(i+c_init))&(df['point_name']==point_name)][['collect_time','point_value']]
            new_name = 'crah_{}_'.format(i+c_init) + point_name
            data_ij.rename(columns = {'point_value':new_name}, inplace = True)
            data_add_time = add_time_idx(data_ij, "collect_time")                          # add time label
            data_add_time[new_name] = delete_outliers(data_add_time[new_name], point_name) # delete outliers
            data_all = pd.concat([data_all, data_add_time[new_name]], axis=1)

    return data_all
    # pd.DataFrame(data_all).to_csv('/home/lpw/time_series/data_all/data{}_all.csv'.format(k))

def collect_data(k):
    # 218 variables: add time label + delete outliers
    if k<=6 or k>=10: 
        data_k = collect_point(k)
    else:
        data_k = collect_point(k,0,17,40,59)

    # forward and backward fill 前向后向填充
    data_fill_f = data_k.fillna(method='ffill')
    data_fill = data_fill_f.fillna(method='bfill')
    return data_fill

In [64]:
for i in range(0,12):
    data = collect_data(i)
    print(data.shape)
    pd.DataFrame(data).to_csv('/home/lpw/time_series/data_all/data{}_all.csv'.format(i))

/tmp/ipykernel_2401/1320076624.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data_folder/data_{}.csv".format(k))
/tmp/ipykernel_2401/692996384.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[(data<0.5)|(data>80)] = np.nan
/tmp/ipykernel_2401/692996384.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[(data<0.5)|(data>80)] = np.nan
/tmp/ipykernel_2401/692996384.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

(25920, 219)


/tmp/ipykernel_2401/1320076624.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data_folder/data_{}.csv".format(k))
/tmp/ipykernel_2401/692996384.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[(data<0.5)|(data>80)] = np.nan
/tmp/ipykernel_2401/692996384.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[(data<0.5)|(data>80)] = np.nan
/tmp/ipykernel_2401/692996384.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

(25920, 219)


/tmp/ipykernel_2401/1320076624.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data_folder/data_{}.csv".format(k))
/tmp/ipykernel_2401/692996384.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[(data<0.5)|(data>80)] = np.nan
/tmp/ipykernel_2401/692996384.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[(data<0.5)|(data>80)] = np.nan
/tmp/ipykernel_2401/692996384.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

...

(25920, 219)


/tmp/ipykernel_2401/1320076624.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data_folder/data_{}.csv".format(k))
/tmp/ipykernel_2401/692996384.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[(data<0.5)|(data>80)] = np.nan
/tmp/ipykernel_2401/692996384.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[(data<0.5)|(data>80)] = np.nan
/tmp/ipykernel_2401/692996384.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

(25920, 219)


/tmp/ipykernel_2401/1320076624.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data_folder/data_{}.csv".format(k))
/tmp/ipykernel_2401/692996384.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[(data<0.5)|(data>80)] = np.nan
/tmp/ipykernel_2401/692996384.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[(data<0.5)|(data>80)] = np.nan
/tmp/ipykernel_2401/692996384.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

(25920, 219)


In [63]:
# # sensor part
# def collect_sensor(k,init=18,end=35):
#     df = pd.read_csv("./data_folder/data_{}.csv".format(k))
#     num = end+1-init
#     for i in range(init,end+1):
#         data_i = df[df['device_cmdb_id']=='sensor_{}'.format(i)]['point_value'][:25000]
#         if i == init: data_sensor = data_i
#         else: 
#             data_sensor = np.vstack((data_sensor,data_i))
#     # print(data_sensor.shape)
#     return data_sensor

# # crah part
# # all_point_name = ['回风温度', '送风温度', '水路进水温度', '系统运行状态', '系统故障状态', '风机运行状态', '风机转速', '制冷状态', '水阀开度', '远程关机']
# all_point_name = ['回风温度', '送风温度', '水路进水温度', '风机转速', '水阀开度'] # continious
# def collect_crah(k,init=20,end=39):
#     df = pd.read_csv("./data_folder/data_{}.csv".format(k))
#     point_num = len(all_point_name)
#     crah_num = end+1-init
#     for j in range(point_num):
#         point_name = all_point_name[j]
#         for i in range(crah_num):
#             data_ij = df[(df['device_cmdb_id']=='crah_{}'.format(i+init))&(df['point_name']==point_name)]['point_value'][:25000]
#             if i==0 and j==0: data_crah = data_ij
#             else: 
#                 data_crah = np.vstack((data_crah,data_ij))
#         # print(data_crah.shape)
#     return data_crah

# def collect_data(k):
#     if k<=6 or k>=10: 
#         d_sensor = collect_sensor(k)
#         d_crah = collect_crah(k)
#     else:
#         d_sensor = collect_sensor(k,0,17)
#         d_crah = collect_crah(k,40,59)
#     all_data = np.vstack((d_sensor,d_crah))
#     return all_data.T
        

In [ ]:
data0_cont = collect_data(0)
print(data0_cont.shape)

In [ ]:

pd.DataFrame(data0_cont).to_csv('data0_cont.csv')

In [ ]:
example = pd.read_csv('data0.csv',index_col=0)
example = np.asarray(example)
print(example.shape)

In [10]:
# 测试每个变量的数量
# sensor part
def collect_sensor(k,init=18,end=35):
    df = pd.read_csv("./data_folder/data_{}.csv".format(k))
    num = end+1-init
    for i in range(init,end+1):
        data_i = df[df['device_cmdb_id']=='sensor_{}'.format(i)]['collect_time']#['point_value']
        print('sensor_{}:'.format(i),len(data_i))
    #     if i == init: data_sensor = data_i
    #     else: 
    #         data_sensor = np.vstack((data_sensor,data_i))
    # print(data_sensor.shape)
    # return data_sensor

collect_sensor(0)

/tmp/ipykernel_25292/1277207524.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data_folder/data_{}.csv".format(k))


sensor_18: 25758
sensor_19: 25758
sensor_20: 25757
sensor_21: 25759
sensor_22: 25760
sensor_23: 25758
sensor_24: 25754
sensor_25: 25758
sensor_26: 25759
sensor_27: 25758
sensor_28: 25758
sensor_29: 25754
sensor_30: 25759
sensor_31: 25755
sensor_32: 25758
sensor_33: 25754
sensor_34: 25755
sensor_35: 25758


In [11]:
all_point_name = ['回风温度', '送风温度', '水路进水温度', '系统运行状态', '系统故障状态', '风机运行状态', '风机转速', '制冷状态', '水阀开度', '远程关机']
def collect_crah(k,init=20,end=39):
    df = pd.read_csv("./data_folder/data_{}.csv".format(k))
    point_num = len(all_point_name)
    crah_num = end+1-init
    for j in range(point_num):
        point_name = all_point_name[j]
        for i in range(crah_num):
            data_ij = df[(df['device_cmdb_id']=='crah_{}'.format(i+init))&(df['point_name']==point_name)]['point_value']
            print(point_name,'crah_{}:'.format(i+init),len(data_ij))
collect_crah(0)

/tmp/ipykernel_25292/1184738457.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data_folder/data_{}.csv".format(k))


回风温度 crah_20: 25758
回风温度 crah_21: 25760
回风温度 crah_22: 25757
回风温度 crah_23: 25758
回风温度 crah_24: 25757
回风温度 crah_25: 25758
回风温度 crah_26: 25754
回风温度 crah_27: 25758
回风温度 crah_28: 25755
回风温度 crah_29: 25758
回风温度 crah_30: 25757
回风温度 crah_31: 25758
回风温度 crah_32: 25754
回风温度 crah_33: 25758
回风温度 crah_34: 25757
回风温度 crah_35: 25758
回风温度 crah_36: 25759
回风温度 crah_37: 25757
回风温度 crah_38: 25758
回风温度 crah_39: 25757
送风温度 crah_20: 25758
送风温度 crah_21: 25760
送风温度 crah_22: 25757
送风温度 crah_23: 25758
送风温度 crah_24: 25757
送风温度 crah_25: 25758
送风温度 crah_26: 25754
送风温度 crah_27: 25758
送风温度 crah_28: 25755
送风温度 crah_29: 25758
送风温度 crah_30: 25757
送风温度 crah_31: 25758
送风温度 crah_32: 25754
送风温度 crah_33: 25758
送风温度 crah_34: 25757
送风温度 crah_35: 25758
送风温度 crah_36: 25759
送风温度 crah_37: 25757
送风温度 crah_38: 25758
送风温度 crah_39: 25757
水路进水温度 crah_20: 25758
水路进水温度 crah_21: 25760
水路进水温度 crah_22: 25757
水路进水温度 crah_23: 25758
水路进水温度 crah_24: 25757
水路进水温度 crah_25: 25758
水路进水温度 crah_26: 25754
水路进水温度 crah_27: 25758
水路进水温度 crah_28: 25755
水路